In [1]:
import json

records = []
for line in open('data/en-US.jsonl', 'r'):
    records.append(json.loads(line))


In [2]:
set([r["intent"] for r in records])

{'alarm_query',
 'alarm_remove',
 'alarm_set',
 'audio_volume_down',
 'audio_volume_mute',
 'audio_volume_other',
 'audio_volume_up',
 'calendar_query',
 'calendar_remove',
 'calendar_set',
 'cooking_query',
 'cooking_recipe',
 'datetime_convert',
 'datetime_query',
 'email_addcontact',
 'email_query',
 'email_querycontact',
 'email_sendemail',
 'general_greet',
 'general_joke',
 'general_quirky',
 'iot_cleaning',
 'iot_coffee',
 'iot_hue_lightchange',
 'iot_hue_lightdim',
 'iot_hue_lightoff',
 'iot_hue_lighton',
 'iot_hue_lightup',
 'iot_wemo_off',
 'iot_wemo_on',
 'lists_createoradd',
 'lists_query',
 'lists_remove',
 'music_dislikeness',
 'music_likeness',
 'music_query',
 'music_settings',
 'news_query',
 'play_audiobook',
 'play_game',
 'play_music',
 'play_podcasts',
 'play_radio',
 'qa_currency',
 'qa_definition',
 'qa_factoid',
 'qa_maths',
 'qa_stock',
 'recommendation_events',
 'recommendation_locations',
 'recommendation_movies',
 'social_post',
 'social_query',
 'takeaway_o

In [3]:
INTENT = "recommendation_movies"
examples = [r["utt"] for r in records if r["intent"] == INTENT]
print(len(examples))
examples[:5]

102


['should i watch this movie',
 'what film should i watch',
 'what is a good iranian film from two thousand and sixteen',
 'what is a good movie to see right now',
 'top ten movies']

In [4]:
INTENTS = ["alarm_set", "audio_volume_down", "datetime_query", "general_greet", "iot_hue_lightoff", "weather_query", "recommendation_movies"]
len(INTENTS)

7

In [5]:
intent2records = {i:[] for i in INTENTS}
for r in records:
    intent = r["intent"]
    if intent in INTENTS:
        x = intent2records[intent]
        x.append(r)

num_records = [len(y) for x,y in intent2records.items()]
num_train = [int(x * 0.7) for x in num_records]
num_test = [x - y for x,y in zip(num_records, num_train)]

num_records, num_train, num_test

([254, 71, 502, 28, 213, 855, 102],
 [177, 49, 351, 19, 149, 598, 71],
 [77, 22, 151, 9, 64, 257, 31])

In [6]:
num_train = {x:y for x,y in zip(INTENTS, num_train)}
num_train

{'alarm_set': 177,
 'audio_volume_down': 49,
 'datetime_query': 351,
 'general_greet': 19,
 'iot_hue_lightoff': 149,
 'weather_query': 598,
 'recommendation_movies': 71}

In [7]:
train = {x:[z["utt"] for z in intent2records[x][:y]] for x,y in num_train.items()}
test = {x:[z["utt"] for z in intent2records[x][y:]] for x,y in num_train.items()}
train

{'alarm_set': ['wake me up at five am this week',
  'wake me up at nine am on friday',
  'set an alarm for two hours from now',
  'set an alarm for twelve',
  'set an alarm forty minutes from now',
  'set alarm for eight every weekday',
  'set alarm at ten am',
  'set alarm tomorrow at six am',
  'set wake up thursday seven am',
  'set an alarm at six in the morning',
  'set an alarm for four in the afternoon',
  'olly alert me at three p. m. to go to the concert',
  'alert me at three p. m. to go to the concert',
  'set an alarm for me at eight am',
  'make an alarm for four p. m.',
  'make me a wake up alarm for eight forty five am',
  'set an alarm for nine am',
  'set a wake up call for ten am',
  'i need a reminder alarm at one',
  'make an alarm for three p. m.',
  'turn on an alarm for nine am',
  'set an alarm for one p. m.',
  'start alarm',
  'set alarm to begin',
  'change alarm to start at midnight',
  'i want to wake up at six am tomorrow please',
  'please set the alarm f

In [8]:
test["weather_query"]

['what is home town weather',
 'how hot is today',
 'how cold is today',
 'what is the weather for this week',
 'is it rainy in my area',
 "what's the weather now",
 'status of weather',
 'what was the weather for the week',
 "how is this week's weather",
 'it seems cold',
 'do i need a coat',
 'what is the forecast for saturday',
 'do i need a raincoat day after',
 'what will be the weather day after',
 "what's the temperature outside",
 'is it hot or cold outside',
 'could you please tell me what is the weather predication for this week',
 'please tell what would be the weather in washington d. c. for the coming week',
 'please tell me what will be rainy day based on the weather forecast',
 'will it rain later in manila',
 'do i need to bring an umbrella to manila this afternoon',
 "what's the weather for the entire week",
 'what will the weather be like for this week',
 'what kind of weather should i expect this week',
 'do i need umbrella now',
 'check if i need umbrella now',
 'is

In [9]:
nlu_yaml = {"version": "3.1"}
nlu_examples = []
for intent, ex in train.items():
    example = {}
    example["intent"] = intent
    example["examples"] = ex
    nlu_examples.append(example)

nlu_yaml["nlu"] = nlu_examples
# nlu_yaml

NameError: name 'nlu_yaml' is not defined

In [ ]:
import yaml
# This doesn't give quite the result we want but fairly close to where a quick bit of hand editting gets the rest of the way there
with open('data/nlu.yml', 'w') as f:
    yaml.dump(nlu_yaml, f, default_flow_style=False)